# Project 02 — NDVI Vegetation Health Analysis (Denpasar)

This notebook computes NDVI from Sentinel-2 (B4,B8), masks clouds (SCL if available), classifies NDVI, and exports:

- NDVI GeoTIFF
- NDVI Classified GeoTIFF (1–4)
- PNG map with discrete colorbar
- Vector polygons (GPKG)
- QA JSON summary

**Instructions**: put a Sentinel-2 GeoTIFF (B4,B8[,SCL]) into `data/raw/` then run cells top → bottom.

In [ ]:
import os, time, json, warnings
warnings.filterwarnings('ignore')
import numpy as np
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.features import shapes
from shapely.geometry import shape
from scipy import ndimage

def safe_remove(path, wait=0.12, retries=8):
    if os.path.exists(path):
        for i in range(retries):
            try:
                os.remove(path)
                return True
            except PermissionError:
                time.sleep(wait)
        raise PermissionError(f'Could not remove {path}. Close apps that lock files.')
    return False

print('Imports OK')
print('rasterio', rasterio.__version__)
print('geopandas', gpd.__version__)
print('numpy', np.__version__)


In [ ]:
BASE = os.path.abspath(os.path.join(os.getcwd(), '..'))
RAW = os.path.join(BASE, 'data', 'raw')
PROC = os.path.join(BASE, 'data', 'processed')
OUT_MAPS = os.path.join(BASE, 'outputs', 'maps')
OUT_SHAPES = os.path.join(BASE, 'outputs', 'shapefiles')
for p in [RAW, PROC, OUT_MAPS, OUT_SHAPES]:
    os.makedirs(p, exist_ok=True)
print('BASE =', BASE)
print('RAW =', RAW)


In [ ]:
candidates = [f for f in os.listdir(RAW) if f.lower().endswith('.tif') and ('sentinel' in f.lower() or 's2' in f.lower())]
if not candidates:
    raise FileNotFoundError('No Sentinel TIFF found in data/raw/. Export B4,B8,SCL from GEE and place here.')
sentinel_path = os.path.join(RAW, candidates[0])
print('Using file:', sentinel_path)


In [ ]:
with rasterio.open(sentinel_path) as src:
    print('CRS:', src.crs)
    print('Width,Height,Count:', src.width, src.height, src.count)
    print('Dtypes:', src.dtypes)
    meta = src.meta.copy()

with rasterio.open(sentinel_path) as src:
    if src.count < 2:
        raise ValueError('TIFF must contain at least two bands: B4 and B8.')
    b4 = src.read(1).astype('float32')  # RED
    b8 = src.read(2).astype('float32')  # NIR
    scl = src.read(3).astype('int16') if src.count >= 3 else None
    transform = src.transform
    crs = src.crs

print('Band shapes:', b4.shape, b8.shape)
if scl is not None:
    print('SCL sample unique:', np.unique(scl)[:10])


In [ ]:
# If values look like reflectance ×10000, scale to 0..1
if np.nanmax(b4) > 2 or np.nanmax(b8) > 2:
    print('Scaling reflectance by 10000')
    b4 = b4 / 10000.0
    b8 = b8 / 10000.0
print('b4 range:', np.nanmin(b4), np.nanmax(b4))
print('b8 range:', np.nanmin(b8), np.nanmax(b8))


In [ ]:
def mask_clouds_using_scl(scl_arr):
    # SCL codes: 3=shadow, 7=unclassified?, 8..11=clouds/cirrus/snow
    cloud_codes = [3,7,8,9,10,11]
    return ~np.isin(scl_arr, cloud_codes)

if scl is not None:
    good_mask = mask_clouds_using_scl(scl)
else:
    # fallback: valid finite and >0
    good_mask = np.isfinite(b4) & np.isfinite(b8) & (b4 > 0) & (b8 > 0)

b4m = np.where(good_mask, b4, np.nan)
b8m = np.where(good_mask, b8, np.nan)
print('Good pixels:', np.sum(~np.isnan(b4m)))


In [ ]:
ndvi = (b8m - b4m) / (b8m + b4m + 1e-8)
ndvi = np.clip(ndvi, -1, 1)

# Use nodata numeric value for compatibility
ndvi_meta = meta.copy()
ndvi_meta.update({'count':1, 'dtype':'float32', 'nodata': -9999.0})
ndvi_path = os.path.join(PROC, 'ndvi_denpasar.tif')
if os.path.exists(ndvi_path): safe_remove(ndvi_path)

ndvi_out = np.nan_to_num(ndvi, nan=-9999.0).astype('float32')
with rasterio.open(ndvi_path, 'w', **ndvi_meta) as dst:
    dst.write(ndvi_out, 1)

print('Saved NDVI ->', ndvi_path)


In [ ]:
valid = ndvi[np.isfinite(ndvi)]
print('NDVI min,max,mean:', float(valid.min()), float(valid.max()), float(valid.mean()))
plt.figure(figsize=(6,3))
plt.hist(valid.flatten(), bins=60)
plt.title('NDVI histogram')
plt.show()


In [ ]:
p10, p50, p90 = np.percentile(valid, [10,50,90])
print('p10,p50,p90 =', p10, p50, p90)

ndvi_class = np.zeros(ndvi.shape, dtype='uint8')
ndvi_class[np.isnan(ndvi)] = 0
ndvi_class[ndvi <= p10] = 1
ndvi_class[(ndvi > p10) & (ndvi <= p50)] = 2
ndvi_class[(ndvi > p50) & (ndvi <= p90)] = 3
ndvi_class[ndvi > p90] = 4

print('Class counts:', np.unique(ndvi_class, return_counts=True))


In [ ]:
# Optional: remove small noisy patches (morphology) before saving
clean_mask = ndimage.binary_opening((ndvi_class>0).astype('uint8'), structure=np.ones((3,3))).astype('uint8')
# transfer cleaned mask back to class by keeping largest contiguous class majority per patch is more complex
# For simplicity we keep classes but you can apply more advanced smoothing per-class if needed

class_meta = ndvi_meta.copy(); class_meta.update({'dtype':'uint8', 'nodata':0})
class_path = os.path.join(PROC, 'ndvi_classified.tif')
if os.path.exists(class_path): safe_remove(class_path)
with rasterio.open(class_path, 'w', **class_meta) as dst:
    dst.write(ndvi_class.astype('uint8'), 1)
print('Saved classified TIFF ->', class_path)

import matplotlib.colors as mcolors
colors = ['#d73027', '#fc8d59', '#fee08b', '#1a9850']
cmap = mcolors.ListedColormap(colors)
bounds = [0.5,1.5,2.5,3.5,4.5]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(8,8))
im = ax.imshow(ndvi_class, cmap=cmap, norm=norm)
ax.axis('off')
cbar = fig.colorbar(im, ax=ax, boundaries=bounds, ticks=[1,2,3,4], fraction=0.046, pad=0.04)
cbar.ax.set_yticklabels(['1 Low','2 Moderate','3 High','4 Very High'])
out_png = os.path.join(OUT_MAPS, 'ndvi_classified_map.png')
if os.path.exists(out_png): safe_remove(out_png)
plt.savefig(out_png, dpi=300, bbox_inches='tight')
plt.show()
print('Saved PNG ->', out_png)


In [ ]:
# Polygonize classes and optionally simplify/dissolve
mask_arr = ndvi_class.astype('uint8')
shp_list = []
for geom, val in shapes(mask_arr, mask=mask_arr>0, transform=transform):
    shp_list.append({'geometry': shape(geom), 'class': int(val)})

gdf = gpd.GeoDataFrame(shp_list, crs=crs)
# dissolve by class to reduce number of polygons
gdf = gdf.dissolve(by='class')

# optional: simplify geometries (tolerance in CRS units)
try:
    # if CRS is geographic (deg), simplify small tol — avoid big distortion
    tol = 0.0002 if '4326' in str(crs) else 10
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=tol)
except Exception:
    pass

out_gpkg = os.path.join(OUT_SHAPES, 'ndvi_denpasar_classes.gpkg')
if os.path.exists(out_gpkg): safe_remove(out_gpkg)
gdf.to_file(out_gpkg, driver='GPKG')
print('Saved GPKG ->', out_gpkg)
print('Vector summary:', gdf.geom_type.value_counts())


In [ ]:
report = {
    'ndvi_min': float(valid.min()),
    'ndvi_max': float(valid.max()),
    'ndvi_mean': float(valid.mean()),
    'class_counts': dict(zip(*np.unique(ndvi_class, return_counts=True)))
}
with open(os.path.join(OUT_MAPS, 'ndvi_stats.json'), 'w') as f:
    json.dump(report, f, indent=2)
print('Saved QA JSON ->', os.path.join(OUT_MAPS, 'ndvi_stats.json'))

print('\nFiles written:')
print('processed:', os.listdir(PROC))
print('maps:', os.listdir(OUT_MAPS))
print('shapefiles:', os.listdir(OUT_SHAPES))
